In [1]:
from importlib import reload
import recovar.config
import pandas as pd
import numpy as np
import pickle
import subprocess
import os, sys

from cryodrgn import analysis
from cryodrgn import utils
# from cryodrgn import dataset
from cryodrgn import ctf
from recovar import plot_utils
from recovar import output, dataset

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
from ipywidgets import interact, interactive, HBox, VBox
from scipy.spatial.transform import Rotation as RR
# py.init_notebook_mode()
from IPython.display import FileLink, FileLinks
import jax.numpy as jnp
from recovar.fourier_transform_utils import fourier_transform_utils
from recovar import core
ftu = fourier_transform_utils(jnp)

grid_size =256//2
log_n = 4
output_folder = f"/home/mg6942/mytigress/spike{grid_size}_new{log_n}/"
# output_folder = f"/home/mg6942/mytigress/spike{grid_size}_new3/"
# output_folder ='/home/mg6942/mytigress/spike256/../'
volume_folder_input =  f"/home/mg6942/mytigress/spike{grid_size}/"
outlier_file_input = "/home/mg6942/mytigress/simulated_empiar10180/volumes/vol0915.mrc"
dataset_dict = dataset.get_default_dataset_option()

# Fill these options with the path to preprocessed files
experiment_directory = '/home/mg6942/mytigress/uniform/'
dataset_dict['ctf_file'] = output_folder + "ctf.pkl"
dataset_dict['poses_file'] = output_folder + "poses.pkl"
dataset_dict['particles_file'] = f"{output_folder}particles.{grid_size}.mrcs"

# Returns an object that knows everything about the dataset.
cryo_dataset = dataset.load_dataset_from_dict(dataset_dict, lazy = False)
print("n images", cryo_dataset.n_images)
del dataset_dict['ind']
cryos = dataset.get_split_datasets_from_dict(dataset_dict, dataset.split_index_list(np.arange(cryo_dataset.n_images)))

# cryo = dataset.load_cryodrgn_dataset( f"{output_folder}particles.{grid_size}.mrcs", output_folder + "poses.pkl", output_folder + "ctf.pkl", lazy = False)
from recovar import regularization, synthetic_dataset, noise, homogeneous, utils, adaptive_kernel_discretization
sim_info = recovar.utils.pickle_load(output_folder + '/simulation_info.pkl')
gt_recon = synthetic_dataset.load_heterogeneous_reconstruction(sim_info, volumes_path_root = None)
gt_vol =gt_recon.get_mean()
noise_variance = sim_info['noise_variance']
HH = synthetic_dataset.load_heterogeneous_reconstruction(sim_info, volumes_path_root = None)
gt_mean = HH.get_mean()
signal_variance = utils.make_radial_image(regularization.average_over_shells(np.abs(gt_mean)**2, HH.volume_shape), HH.volume_shape)

downsampled_mean = gt_mean * cryos[0].get_valid_frequency_indices(rad = 10)



(INFO) (xla_bridge.py) (19-Feb-24 13:03:04) Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA
(INFO) (xla_bridge.py) (19-Feb-24 13:03:04) Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(INFO) (config.py) (19-Feb-24 13:03:04) Devices found: NVIDIA A100-SXM4-80GB


2024-02-19 13:03:14.029337: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 13:03:14.029421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 13:03:14.049566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 13:03:16.256145: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(INFO) (ctf.py) (19-Feb-24 13:03:19) Image size (pix)  : 128
(INFO) (ctf.py) (19-Feb-24 13:03:19) A/pix             : 3.0
(INFO) (ctf.py) (19-Feb-24 13:03:19) DefocusU (A)      : 19267.099609375
(INFO) (ctf.py) (19-Feb-24 13:03:19) DefocusV (A)      : 19048.19921875
(INFO) (ctf.py) (19-Feb-24 13:03:19) Dfang (deg)       : 77.30999755859375
(INFO) (ctf.py) (19-Feb-24 13:03:19) voltage (kV)      : 300.0
(INFO) (ctf.py) (19-Feb-24 13:03:19) cs (mm)           : 2.700000047683716
(INFO) (ctf.py) (19-Feb-24 13:03:19) w                 : 0.07000000029802322
(INFO) (ctf.py) (19-Feb-24 13:03:19) Phase shift (deg) : 0.0
n images 10000
(INFO) (ctf.py) (19-Feb-24 13:03:19) Image size (pix)  : 128
(INFO) (ctf.py) (19-Feb-24 13:03:19) A/pix             : 3.0
(INFO) (ctf.py) (19-Feb-24 13:03:19) DefocusU (A)      : 19267.099609375
(INFO) (ctf.py) (19-Feb-24 13:03:19) DefocusV (A)      : 19048.19921875
(INFO) (ctf.py) (19-Feb-24 13:03:19) Dfang (deg)       : 77.30999755859375
(INFO) (ctf.py) (19-Feb-2

In [2]:
import healpy as hp
import numpy as np
NSIDE = 2**1
nside = NSIDE
m = hp.nside2npix(NSIDE)

z = hp.pix2ang(nside, np.arange(m))

angle_res = hp.nside2resol(nside, arcmin=True) / 60 
n_in_planes = np.round(360 / angle_res).astype(int)
in_angle_angles = np.linspace(0, 2 * np.pi, n_in_planes)

angles = np.meshgrid( np.arange(m), in_angle_angles )
theta = z[0][angles[0]]
phi = z[1][angles[0]]
angles = np.stack( [theta, phi, angles[1] ], axis=-1)
angles = angles.reshape( -1, 3)

from cryodrgn import utils as cryodrgn_utils
rotation_matrices = np.array([cryodrgn_utils.R_from_relion(*angle) for angle in angles])


In [ ]:
reload(core)
import jax

import warnings
warnings.filterwarnings("error")

# TODO: Should it be residual of masked?
# Residual will be 4 dimensional
# volumes_batch x images_batch x pose_batch  
# @functools.partial(jax.jit, static_argnums = [7,8,9,10,11,12])    
def compute_residuals_many_poses(volumes, images, rotation_matrices, translations, CTF_params, noise_variance, voxel_size, volume_shape, image_shape, grid_size, disc_type, CTF_fun ):
    # Rotations should be vol_batch x images_batch x rotation_batch x 10
    # assert(rotation_matrices.shape[0] == volumes.shape[0])
    # assert((translations.shape[:-1] == rotation_batch.shape[0]).all())
    # assert(rotation_matrices.shape[1] == images.shape[0])
    # assert(translations.shape[0] == volumes.shape[0])
    # assert(translations.shape[1] == images.shape[0])

    # n_vols x rotations x image_size
    projected_volumes = core.batch_vol_rot_slice_volume_by_map(volumes, rotation_matrices, image_shape, volume_shape, disc_type)
    # Broadcast CTF in volumes x rotations
    projected_volumes = (projected_volumes * CTF_fun( CTF_params, image_shape, voxel_size)[None,:,None,:])[...,None,:]

    # How should translations be fixed ? hmmm
    # Broacast over volumes x rotations
    translated_images = core.batch_trans_translate_images(images, translations, image_shape)[None,:, None]
    
    norm_res_squared = jnp.linalg.norm((projected_volumes - translated_images) / jnp.sqrt(noise_variance), axis = (-1))**2

    # Output is vol_batch x image_batch x rots_batch x trans_batch
    return norm_res_squared


def compute_probability_from_residual_normal_squared_one_image(norm_res_squared):
    exp_res = jnp.exp(- norm_res_squared)
    summed_exp = jnp.sum(exp_res)
    return exp_res / summed_exp

compute_probability_from_residual_normal_squared = jax.vmap(compute_probability_from_residual_normal_squared_one_image)

def compute_adj_slice(one_image, rotation_matrices, translations, CTF_params, probabilities):
    CTF = CTF_fun( CTF_params, image_shape, voxel_size)
    one_image_summed  = jnp.sum(batch_translate(one_image * CTF * probabilities, translations))
    indices = core.get_nearest_gridpoint_indices(rotation_matrix, image_shape, volume_shape, grid_size)
    volume = core.summed_adjoint_slice_by_nearest(volume_size, one_image_summed, indices)
    return volume


cryo = cryos[0]
volumes = gt_mean

noise_variance_rad = noise.make_radial_noise(noise_variance, cryo.image_shape)
batch_size = 10
experiment_dataset = cryo
data_generator = experiment_dataset.get_dataset_generator(batch_size=batch_size) 
disc_type = 'nearest'

rotation_grid = rotation_matrices
translation_grid = (np.ones((5,2)) * 0 )
norm_res_squared_l =  []
for batch, indices in data_generator:
    # Only place where image mask is used ?
    # print('TOOK OUT IMAGE MASK IN MEAN!!! PUT IT BACK??')
    all_rotations = np.repeat(rotation_grid[None], axis=0, repeats = indices.size)[None]
    all_translations = np.repeat(translation_grid[None], axis=0, repeats = indices.size)
    batch = experiment_dataset.image_stack.process_images(batch, apply_image_mask = False)
    norm_res_squared = compute_residuals_many_poses(volumes[None],batch,
                                                      all_rotations,
                                                      all_translations,
                                                    cryo.CTF_params[indices],
                                                      noise_variance_rad, cryo.voxel_size,
                                                      cryo.volume_shape, cryo.image_shape,
                                                      cryo.grid_size, disc_type, cryo.CTF_fun )
    
    
    probs = compute_probability_from_residual_normal_squared(norm_res_squared)
    norm_res_squared_l.append(norm_res_squared)
    import pdb; pdb.set_trace()


> /tmp/ipykernel_1613347/1110154443.py(61)<module>()
     59 translation_grid = (np.ones((5,2)) * 0 )
     60 norm_res_squared_l =  []
---> 61 for batch, indices in data_generator:
     62     # Only place where image mask is used ?
     63     # print('TOOK OUT IMAGE MASK IN MEAN!!! PUT IT BACK??')



In [ ]:
norm_res_squared_l.shape

In [ ]:
norm_res_squared_l = jnp.concatenate(norm_res_squared_l, axis=0)

In [ ]:
norm_res_squared_l.shape

In [ ]:
pdb

In [ ]:
from cryodrgn import utils as cryodrgn_utils
rotation_matrices = np.array([cryodrgn_utils.R_from_relion(*angle) for angle in angles])

In [ ]:
from cryodrgn import rotation

In [ ]:
zz = z[0][angles[0]]


In [ ]:
in_angle_angles.shape

In [ ]:
# plt.scatter3(

In [ ]:
pdb

In [ ]:
import recovar.config
from importlib import reload
import numpy as np
from cryodrgn import analysis
from cryodrgn import utils
from cryodrgn import ctf
from recovar import plot_utils
from recovar import output, dataset
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
from recovar import simulator
reload(simulator)
import jax
import warnings
# warnings.filterwarnings("error")
grid_size =128*1
output_folder = f"/home/mg6942/mytigress/spike{grid_size}_new/"
# output_folder ='/home/mg6942/mytigress/spike256/../'
volume_folder_input =  f"/home/mg6942/mytigress/spike{grid_size}/"
# volume_folder_input = f"/home/mg6942/mytigress/constant_vol/"

outlier_file_input = "/home/mg6942/mytigress/simulated_empiar10180/volumes/vol0915.mrc"
n_images = int(1e2)
voxel_size = 3 * 128 / grid_size#3/2/2
output.mkdir_safe(output_folder)
volume_distribution = np.zeros(1)
first_k = 1
volume_distribution[:first_k] = 1/first_k

image_stack, sim_info = simulator.generate_synthetic_dataset(output_folder, voxel_size, volume_folder_input, 
                                                             outlier_file_input, n_images, grid_size = grid_size,
                               volume_distribution = volume_distribution,  dataset_params_option = "dataset1", noise_level =1e-8, 
                               noise_model = "white", put_extra_particles = False, percent_outliers = 0.0, 
                               volume_radius = 0.6, trailing_zero_format_in_vol_name = True, noise_scale_std = 0.2 * 0, contrast_std =0.1 * 0 , disc_type = 'nufft')